#Лабораторная работа №4
##Оптимизировать алгоритмы градиентного спуска с помощью Optuna

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import optuna
import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection
import logging
import sys
import math
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

Функция Шаффера №2

In [ ]:
def Sh2_E(x,y):
  return 0.5+((math.sin(x**2-y**2))**2-0.5)/(1+0.001*(x**2+y**2))**2
def objective_Sh2(trial):
    x = trial.suggest_float("x", -100, 100)
    y = trial.suggest_float("y", -100, 100)
    return Sh2_E(x, y)

Функция Шаффера №4

In [ ]:
def Sh4_E(x,y):
  return 0.5+((math.cos(math.sin(abs(x**2-y**2))))**2-0.5)/(1+0.001*(x**2+y**2))**2
def objective_Sh4(trial):
    x = trial.suggest_float("x", -100, 100)
    y = trial.suggest_float("y", -5, 5)
    return Sh4_E(x, y)

In [ ]:
study_Sh2 = optuna.create_study()
study_Sh2.optimize(objective_Sh2, n_trials=100)

study_Sh4 = optuna.create_study()
study_Sh4.optimize(objective_Sh4, n_trials=100)

[I 2022-12-28 13:56:07,556] A new study created in memory with name: no-name-c43efe84-b789-4e87-9c1d-f79b7b0fdbed
[I 2022-12-28 13:56:07,561] Trial 0 finished with value: 0.4945665518439392 and parameters: {'x': 23.185936111930943, 'y': 77.89246632503674}. Best is trial 0 with value: 0.4945665518439392.
[I 2022-12-28 13:56:07,570] Trial 1 finished with value: 0.49956473890214254 and parameters: {'x': -99.43563953391859, 'y': 46.56009366999078}. Best is trial 0 with value: 0.4945665518439392.
[I 2022-12-28 13:56:07,575] Trial 2 finished with value: 0.48567514767457587 and parameters: {'x': 65.43654607589139, 'y': 24.749009757780243}. Best is trial 2 with value: 0.48567514767457587.
[I 2022-12-28 13:56:07,580] Trial 3 finished with value: 0.5078121478416683 and parameters: {'x': 54.11903479963726, 'y': -31.256403119249}. Best is trial 2 with value: 0.48567514767457587.
[I 2022-12-28 13:56:07,586] Trial 4 finished with value: 0.4905441614425885 and parameters: {'x': 79.10758108982762, 'y'

Выводим результат

In [ ]:
best_params_GP = study_GP.best_params
found_x = best_params_GP["x"]
found_y = best_params_GP["y"]
print("Функция Гольдшейна-Прайса:")
print(f"x: {found_x}, y: {found_y}, E: {GP_E(found_x, found_y)}")
best_params_H = study_H.best_params
found_x = best_params_H["x"]
found_y = best_params_H["y"]
print("Функция Химмельблау:")
print(f"x: {found_x}, y: {found_y}, E: {H_E(found_x, found_y)}")

Функция Гольдшейна-Прайса:
x: -0.016585135340439733, y: -0.956219127535605, E: 3.9894198326836223
Функция Химмельблау:
x: -2.74501531414102, y: 3.004744729792382, E: 0.7251417697341881


##Часть 2
Проведем оптимизацию градиентного спуска на датасете Diabetes

In [ ]:
def objective(trial):
    BC = sklearn.datasets.load_diabetes()
    classes = list(set(BC.target))
    train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(
        BC.data, BC.target, test_size=0.25, random_state=0)
    alpha = trial.suggest_float("alpha", 1e-5, 1e-1, log=True)
    clf = sklearn.linear_model.SGDClassifier(alpha=alpha)
    for step in range(100):
        clf.partial_fit(train_x, train_y, classes=classes)
        intermediate_value = 1.0 - clf.score(valid_x, valid_y)
        trial.report(intermediate_value, step)
        if trial.should_prune():
            raise optuna.TrialPruned()
    return 1.0 - clf.score(valid_x, valid_y)

In [ ]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20)

[I 2022-11-13 18:07:13,844] A new study created in memory with name: no-name-3ae68bbf-8d38-4f38-91ae-df88fad04afc


A new study created in memory with name: no-name-3ae68bbf-8d38-4f38-91ae-df88fad04afc
A new study created in memory with name: no-name-3ae68bbf-8d38-4f38-91ae-df88fad04afc
A new study created in memory with name: no-name-3ae68bbf-8d38-4f38-91ae-df88fad04afc
A new study created in memory with name: no-name-3ae68bbf-8d38-4f38-91ae-df88fad04afc


[I 2022-11-13 18:07:22,159] Trial 0 finished with value: 1.0 and parameters: {'alpha': 0.00017496238466290127}. Best is trial 0 with value: 1.0.


Trial 0 finished with value: 1.0 and parameters: {'alpha': 0.00017496238466290127}. Best is trial 0 with value: 1.0.
Trial 0 finished with value: 1.0 and parameters: {'alpha': 0.00017496238466290127}. Best is trial 0 with value: 1.0.
Trial 0 finished with value: 1.0 and parameters: {'alpha': 0.00017496238466290127}. Best is trial 0 with value: 1.0.
Trial 0 finished with value: 1.0 and parameters: {'alpha': 0.00017496238466290127}. Best is trial 0 with value: 1.0.


[I 2022-11-13 18:07:30,369] Trial 1 finished with value: 1.0 and parameters: {'alpha': 0.0016142203351917478}. Best is trial 0 with value: 1.0.


...

Trial 17 finished with value: 1.0 and parameters: {'alpha': 0.009919987407471208}. Best is trial 2 with value: 0.990990990990991.
Trial 17 finished with value: 1.0 and parameters: {'alpha': 0.009919987407471208}. Best is trial 2 with value: 0.990990990990991.
Trial 17 finished with value: 1.0 and parameters: {'alpha': 0.009919987407471208}. Best is trial 2 with value: 0.990990990990991.
Trial 17 finished with value: 1.0 and parameters: {'alpha': 0.009919987407471208}. Best is trial 2 with value: 0.990990990990991.


[I 2022-11-13 18:08:45,646] Trial 18 finished with value: 1.0 and parameters: {'alpha': 0.00041391643021731144}. Best is trial 2 with value: 0.990990990990991.


Trial 18 finished with value: 1.0 and parameters: {'alpha': 0.00041391643021731144}. Best is trial 2 with value: 0.990990990990991.
Trial 18 finished with value: 1.0 and parameters: {'alpha': 0.00041391643021731144}. Best is trial 2 with value: 0.990990990990991.
Trial 18 finished with value: 1.0 and parameters: {'alpha': 0.00041391643021731144}. Best is trial 2 with value: 0.990990990990991.
Trial 18 finished with value: 1.0 and parameters: {'alpha': 0.00041391643021731144}. Best is trial 2 with value: 0.990990990990991.


[I 2022-11-13 18:08:54,571] Trial 19 finished with value: 1.0 and parameters: {'alpha': 0.003447198232149919}. Best is trial 2 with value: 0.990990990990991.


Trial 19 finished with value: 1.0 and parameters: {'alpha': 0.003447198232149919}. Best is trial 2 with value: 0.990990990990991.
Trial 19 finished with value: 1.0 and parameters: {'alpha': 0.003447198232149919}. Best is trial 2 with value: 0.990990990990991.
Trial 19 finished with value: 1.0 and parameters: {'alpha': 0.003447198232149919}. Best is trial 2 with value: 0.990990990990991.
Trial 19 finished with value: 1.0 and parameters: {'alpha': 0.003447198232149919}. Best is trial 2 with value: 0.990990990990991.


In [ ]:
plot_optimization_history(study)

In [ ]:
plot_intermediate_values(study)

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_edf(study)